<a href="https://colab.research.google.com/github/TeckniX/dagleash/blob/main/explore_dags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# uncomment to setup GDrive for everything
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# uncomment to clone the repo
#!git clone https://github.com/TeckniX/dagleash.git

fatal: destination path 'dagleash' already exists and is not an empty directory.


In [7]:
# change context to the cloned repo
%cd /content/dagleash

/content/dagleash


In [6]:
DAGSHUB_USER= "lokitek" #@param {type:"string"}
DAGSHUB_REPO_OWNER = "lokitek" #@param {type:"string"}
DAGSHUB_REPO="COCO_1K" #@param {type:"string"}
DAGSHUB_FULL_REPO=DAGSHUB_REPO_OWNER + "/" + DAGSHUB_REPO
DATASOURCE_NAME = "COCO_1K" #@param {type:"string"}
MLFLOW_PROJECT = "Default" #@param {type:"string"}

In [15]:
# Generate random hash for datasource
import hashlib
import secrets

def generate_random_hash(length=32):
    """Generates a random hash of specified length."""

    # Generate random bytes
    random_bytes = secrets.token_bytes(length)

    # Create a hash object (using SHA-256)
    hash_object = hashlib.sha256()

    # Update the hash object with the random bytes
    hash_object.update(random_bytes)

    # Get the hexadecimal representation of the hash
    hex_digest = hash_object.hexdigest()

    return hex_digest

In [8]:
# setup imports
!pip install --upgrade --quiet dagshub torch mlflow ultralytics label_studio_converter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/

In [18]:
!pip freeze > requirements.txt

In [16]:
# uncomment to download the latest data
#!dagshub download Dean/COCO_1K data --keep-prefix

In [10]:
import yaml
import torch

from utils.config import Config
from utils.dagshub_yolo_cb import custom_callbacks_fn
from utils.data import DataFunctions

import dagshub
from dagshub.data_engine import datasources, datasets

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [14]:
classes = Config.classes
dataset_func = DataFunctions(dataset_dir="data/", classes=classes, label_type='segmentation')

#DATASOURCE_NAME = DATASOURCE_NAME + "_" + generate_random_hash()

ds = datasources.create_from_repo(DAGSHUB_FULL_REPO, DATASOURCE_NAME, "data/images")
# ds = datasources.get_datasource(DAGSHUB_FULL_REPO, DATASOURCE_NAME)
# ds.all().download_binary_columns().dataframe
# print(ds.head().dataframe)

Output()

Empty DataFrame
Columns: []
Index: []


In [17]:
md_query = ds.all()
md = md_query.dataframe
# print(md)

enriched_md = md.apply(lambda x: dataset_func.create_metadata(x), axis=1)
# print(enriched_md)

Output()

                        path  datapoint_id  \
0     train/000000000009.jpg      81625544   
1     train/000000000025.jpg      81625545   
2     train/000000000030.jpg      81625546   
3     train/000000000034.jpg      81625547   
4     train/000000000036.jpg      81625548   
...                      ...           ...   
1194    val/000000109827.jpg      81626738   
1195    val/000000110042.jpg      81626739   
1196    val/000000112997.jpg      81626740   
1197    val/000000121673.jpg      81626741   
1198    val/000000123633.jpg      81626742   

                                   dagshub_download_url  height  media type  \
0     https://dagshub.com/api/v1/repos/lokitek/COCO_...     480  image/jpeg   
1     https://dagshub.com/api/v1/repos/lokitek/COCO_...     426  image/jpeg   
2     https://dagshub.com/api/v1/repos/lokitek/COCO_...     428  image/jpeg   
3     https://dagshub.com/api/v1/repos/lokitek/COCO_...     425  image/jpeg   
4     https://dagshub.com/api/v1/repos/lokitek/COCO_

In [19]:
dagshub.common.config.dataengine_metadata_upload_batch_size = 50
ds.upload_metadata_from_dataframe(enriched_md, path_column="path")

Output()

In [20]:
# Find data with the category of giraffe
giraffe_ds = (ds['categories'].contains('giraffe')).head()
giraffe_ds.dataframe

Output()

,path,datapoint_id,dagshub_download_url,annotation,categories,height,media type,size,split,valid_datapoint,width,year
0,train/000000000025.jpg,81625545,https://dagshub.com/api/v1/repos/lokitek/COCO_...,7ff2628c653ee993968bbf534b83d29b1ca491b787ef49...,giraffe,426,image/jpeg,196370,train,True,640,2017
1,train/000000000072.jpg,81625554,https://dagshub.com/api/v1/repos/lokitek/COCO_...,4081872103f6abd54a0b47ba4a1d6856ca0f85cf944f99...,giraffe,640,image/jpeg,239116,train,True,427,2017
2,train/000000000136.jpg,81625569,https://dagshub.com/api/v1/repos/lokitek/COCO_...,e432066525d8921dd199df0eae69e5c142920d0f0f4636...,"person, giraffe",374,image/jpeg,105082,train,True,500,2017
3,train/000000000144.jpg,81625573,https://dagshub.com/api/v1/repos/lokitek/COCO_...,1ed71d3e02853a8c7c41e86d9300eecd2858c3855fade1...,giraffe,480,image/jpeg,225663,train,True,640,2017
4,train/000000000706.jpg,81625685,https://dagshub.com/api/v1/repos/lokitek/COCO_...,0d07c321f48b0b11b18223e98c920c64b34c6dea0d0bdd...,giraffe,640,image/jpeg,196707,train,True,428,2017
5,train/000000000753.jpg,81625695,https://dagshub.com/api/v1/repos/lokitek/COCO_...,b36377e3ab9b35c3eabd60d312de848890292f18ccffa2...,giraffe,640,image/jpeg,266443,train,True,426,2017
6,train/000000000996.jpg,81625744,https://dagshub.com/api/v1/repos/lokitek/COCO_...,4d54d0b3e295df55b88d06b7797c0abec7a4f9c53cdb63...,giraffe,640,image/jpeg,303199,train,True,480,2017
7,train/000000001063.jpg,81625754,https://dagshub.com/api/v1/repos/lokitek/COCO_...,5bb7fb0db92b3757dee7749766f3acac9bd6da36bae94c...,giraffe,640,image/jpeg,267297,train,True,480,2017
8,train/000000001224.jpg,81625790,https://dagshub.com/api/v1/repos/lokitek/COCO_...,deb077e675ab2551f69c0ecdeebf62c36a76387246004d...,giraffe,427,image/jpeg,220230,train,True,640,2017
9,train/000000001282.jpg,81625800,https://dagshub.com/api/v1/repos/lokitek/COCO_...,c7118dcce4b9c1e4724cfd4812e6c99a8ed445b6620553...,giraffe,640,image/jpeg,238627,train,True,426,2017


In [21]:
giraffe_ds.visualize(visualizer='dagshub')

The visualization is available at the following link:
https://dagshub.com/lokitek/COCO_1K/datasets/datasource/4146/gallery?query=eyJxdWVyeSI6IHsiZmlsdGVyIjogeyJrZXkiOiAiY2F0ZWdvcmllcyIsICJ2YWx1ZSI6ICJnaXJhZmZlIiwgInZhbHVlVHlwZSI6ICJTVFJJTkciLCAiY29tcGFyYXRvciI6ICJDT05UQUlOUyJ9fX0=


'https://dagshub.com/lokitek/COCO_1K/datasets/datasource/4146/gallery?query=eyJxdWVyeSI6IHsiZmlsdGVyIjogeyJrZXkiOiAiY2F0ZWdvcmllcyIsICJ2YWx1ZSI6ICJnaXJhZmZlIiwgInZhbHVlVHlwZSI6ICJTVFJJTkciLCAiY29tcGFyYXRvciI6ICJDT05UQUlOUyJ9fX0='

In [22]:
# find the data with category of cake
q = ds['categories'].contains("cake")
q.head().dataframe

Output()

,path,datapoint_id,dagshub_download_url,annotation,categories,height,media type,size,split,valid_datapoint,width,year
0,train/000000000092.jpg,81625562,https://dagshub.com/api/v1/repos/lokitek/COCO_...,86d6151007613e1e1701df8a2d86908aafcf06421c06d1...,"fork, cake",427,image/jpeg,73926,train,True,640,2017
1,train/000000000113.jpg,81625566,https://dagshub.com/api/v1/repos/lokitek/COCO_...,19fe64697328f50a9c94f0c42bf04c1171c269d62d4224...,"knife, person, chair, cup, cake, dining table",640,image/jpeg,256369,train,True,416,2017
2,train/000000000127.jpg,81625567,https://dagshub.com/api/v1/repos/lokitek/COCO_...,64e11eaadc3b5ec883138f261525c2d0e5682bb067b28b...,"knife, bench, person, cup, book, handbag, umbr...",481,image/jpeg,205469,train,True,640,2017
3,train/000000000428.jpg,81625620,https://dagshub.com/api/v1/repos/lokitek/COCO_...,69aa839c24dcec620c74e0ee220cf61a2d41881cc7adc3...,"tie, person, cake",360,image/jpeg,102365,train,True,640,2017
4,train/000000000735.jpg,81625694,https://dagshub.com/api/v1/repos/lokitek/COCO_...,6071b5f0f29d8a0804809b9a445f99a9c60156be8eda92...,"knife, person, bowl, cake, dining table",361,image/jpeg,135509,train,True,640,2017
5,train/000000000790.jpg,81625704,https://dagshub.com/api/v1/repos/lokitek/COCO_...,0d4c844418ed6c4b8ce820949107e643193a969b8fcb9f...,"knife, clock, person, chair, cup, bowl, fork, ...",422,image/jpeg,209560,train,True,640,2017
6,train/000000000982.jpg,81625740,https://dagshub.com/api/v1/repos/lokitek/COCO_...,f990035aa4071254bf972b2858b9339046f6d92371e520...,cake,640,image/jpeg,158951,train,True,480,2017
7,train/000000001180.jpg,81625783,https://dagshub.com/api/v1/repos/lokitek/COCO_...,d5a09b6597ef3bbbc9775d96b702f8aa4102a0a3154b08...,"knife, person, chair, bowl, fork, cake, dining...",640,image/jpeg,119366,train,True,426,2017
8,train/000000001261.jpg,81625796,https://dagshub.com/api/v1/repos/lokitek/COCO_...,7d74785fe922dbab3f35c90eb08f17bf489042f5574c3e...,"bowl, cake, dining table",480,image/jpeg,230947,train,True,640,2017
9,train/000000001290.jpg,81625801,https://dagshub.com/api/v1/repos/lokitek/COCO_...,820d7bd2f506f24d8d0df3e14a3d2f1cd5486a5812584c...,"person, chair, cup, cake, dining table",540,image/jpeg,184970,train,True,640,2017


In [23]:
q.visualize()

The visualization is available at the following link:
https://dagshub.com/lokitek/COCO_1K/datasets/datasource/4146/gallery?query=eyJxdWVyeSI6IHsiZmlsdGVyIjogeyJrZXkiOiAiY2F0ZWdvcmllcyIsICJ2YWx1ZSI6ICJjYWtlIiwgInZhbHVlVHlwZSI6ICJTVFJJTkciLCAiY29tcGFyYXRvciI6ICJDT05UQUlOUyJ9fX0=


'https://dagshub.com/lokitek/COCO_1K/datasets/datasource/4146/gallery?query=eyJxdWVyeSI6IHsiZmlsdGVyIjogeyJrZXkiOiAiY2F0ZWdvcmllcyIsICJ2YWx1ZSI6ICJjYWtlIiwgInZhbHVlVHlwZSI6ICJTVFJJTkciLCAiY29tcGFyYXRvciI6ICJDT05UQUlOUyJ9fX0='

In [33]:
q.all().annotate()
dl = q.all().as_ml_dataloader(flavor="torch")

Output()

Output()

Open the following link to start working on your annotation project:
https://dagshub.com/lokitek/COCO_1K/annotations/de/send-to-annotation?datapointsUuid=7406f178-91dc-4d5e-9bea-879ada4293cf


[tensor([[[[ 93.,  96.,  95.,  ..., 101., 101., 101.],
           [ 95.,  97.,  96.,  ...,  89.,  90.,  89.],
           [ 96.,  98.,  97.,  ...,  74.,  76.,  74.],
           ...,
           [111., 110., 110.,  ..., 141., 150., 160.],
           [110., 110., 111.,  ..., 152., 163., 173.],
           [109., 111., 112.,  ..., 165., 177., 185.]],
 
          [[ 62.,  65.,  64.,  ...,  91.,  91.,  89.],
           [ 64.,  66.,  65.,  ...,  80.,  78.,  77.],
           [ 65.,  67.,  66.,  ...,  66.,  65.,  63.],
           ...,
           [105., 104., 104.,  ..., 134., 141., 153.],
           [104., 104., 105.,  ..., 146., 155., 164.],
           [103., 105., 106.,  ..., 156., 168., 175.]],
 
          [[  0.,   0.,   0.,  ...,  79.,  79.,  77.],
           [  0.,   1.,   0.,  ...,  65.,  64.,  63.],
           [  0.,   2.,   1.,  ...,  47.,  47.,  45.],
           ...,
           [119., 118., 118.,  ..., 141., 142., 147.],
           [118., 118., 119.,  ..., 150., 152., 157.],
           

In [ ]:
next(iter(dl))

In [34]:
# Query to only get non-new-datapoints
ds = datasources.get_datasource(DAGSHUB_FULL_REPO, DATASOURCE_NAME)
ds = ~(ds['path'].contains("new_data"))
ds.head().dataframe

Output()

,path,datapoint_id,dagshub_download_url,annotation,categories,height,media type,size,split,valid_datapoint,width,year
0,train/000000000009.jpg,81625544,https://dagshub.com/api/v1/repos/lokitek/COCO_...,5337af8fa679ba0d98317b42fde819e7c1dab9c9e87b37...,"broccoli, orange, bowl",480,image/jpeg,224297,train,True,640,2017
1,train/000000000025.jpg,81625545,https://dagshub.com/api/v1/repos/lokitek/COCO_...,7ff2628c653ee993968bbf534b83d29b1ca491b787ef49...,giraffe,426,image/jpeg,196370,train,True,640,2017
2,train/000000000030.jpg,81625546,https://dagshub.com/api/v1/repos/lokitek/COCO_...,e3a001786a79c96a8d6f45e09b3063b1549aa35211ad2d...,"potted plant, vase",428,image/jpeg,71463,train,True,640,2017
3,train/000000000034.jpg,81625547,https://dagshub.com/api/v1/repos/lokitek/COCO_...,65638d764ef991fa69d8145f541af01f16caee56f0524e...,zebra,425,image/jpeg,406018,train,True,640,2017
4,train/000000000036.jpg,81625548,https://dagshub.com/api/v1/repos/lokitek/COCO_...,967eee5bb6a6cfa80fbba58563b4d9bbd6c73759d06a5b...,"person, umbrella",640,image/jpeg,260207,train,True,481,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
95,train/000000000520.jpg,81625639,https://dagshub.com/api/v1/repos/lokitek/COCO_...,51a95b38c907699d9e4c52d77bf784e2a21d1f1347ea15...,"bird, person",480,image/jpeg,105015,train,True,640,2017
96,train/000000000529.jpg,81625640,https://dagshub.com/api/v1/repos/lokitek/COCO_...,d2b3dfc51bda617c75af2f3f373e6bde0406b669637127...,"motorcycle, person",640,image/jpeg,228538,train,True,427,2017
97,train/000000000531.jpg,81625641,https://dagshub.com/api/v1/repos/lokitek/COCO_...,727cc77eae26011af4571990074d0170a775f0b2a7b767...,"tennis racket, person, bicycle, sports ball",480,image/jpeg,94873,train,True,640,2017
98,train/000000000532.jpg,81625642,https://dagshub.com/api/v1/repos/lokitek/COCO_...,3909dcc479fd2b13d0130f7044a8254cbbbe220befcc92...,"handbag, bus, person",480,image/jpeg,214068,train,True,640,2017


In [35]:
ds.head().visualize(visualizer='dagshub')

Output()

The visualization is available at the following link:
https://dagshub.com/lokitek/COCO_1K/datasets/datasource/4146/gallery?query=eyJxdWVyeSI6IHsiZmlsdGVyIjogeyJrZXkiOiAicGF0aCIsICJ2YWx1ZSI6ICJuZXdfZGF0YSIsICJ2YWx1ZVR5cGUiOiAiU1RSSU5HIiwgImNvbXBhcmF0b3IiOiAiQ09OVEFJTlMifSwgIm5vdCI6IHRydWV9fQ==


'https://dagshub.com/lokitek/COCO_1K/datasets/datasource/4146/gallery?query=eyJxdWVyeSI6IHsiZmlsdGVyIjogeyJrZXkiOiAicGF0aCIsICJ2YWx1ZSI6ICJuZXdfZGF0YSIsICJ2YWx1ZVR5cGUiOiAiU1RSSU5HIiwgImNvbXBhcmF0b3IiOiAiQ09OVEFJTlMifSwgIm5vdCI6IHRydWV9fQ=='

In [36]:
# Train model v1
dataset_func.create_yolo_v8_dataset_yaml(ds)
# ultralytics.utils.callbacks.add_integration_callbacks = custom_callbacks_fn

#dataset.all().get_blob_fields("annotation")

Output()

Output()

Output()

Output()

AttributeError: 'PosixPath' object has no attribute 'to_ls_task'

In [37]:
dagshub.init(repo_name=DAGSHUB_REPO, repo_owner=DAGSHUB_USER)

# Load a model
model = ultralytics.YOLO('yolov8n-seg.pt', task='segment')  # load a pretrained model (recommended for training)

with mlflow.start_run():

    # Train the model
    model.train(data='custom_coco.yaml', epochs=1, imgsz=640, device='0', project=MLFLOW_PROJECT)

Initialized MLflow to track repo "lokitek/COCO_1K"

Repository lokitek/COCO_1K initialized!

NameError: name 'ultralytics' is not defined

In [38]:
# add more data
!mkdir data/images/train/new_data && cp -r new_data/* data/images/train/new_data


mkdir: cannot create directory ‘data/images/train/new_data’: File exists


In [39]:
import dagshub
dagshub.upload_files(repo=DAGSHUB_FULL_REPO, local_path="data/images/train/new_data", remote_path="data/images/train/new_data", commit_message="Add new data", force=True)


Output()

Upload successful, content was identical and no new commit was created

Directory upload complete, uploaded 100 files to 
https://dagshub.com/lokitek/COCO_1K/src/main/data%2Fimages%2Ftrain%2Fnew_data

In [40]:
# scan the new data and set it as a ds
from dagshub.data_engine import datasources
ds = datasources.get_datasource(DAGSHUB_FULL_REPO, DATASOURCE_NAME)

new_data_q = (ds["path"].contains("new_data"))
new_data = new_data_q.all()
new_md = new_data.dataframe
new_md

Output()

,path,datapoint_id,dagshub_download_url,annotation,categories,height,media type,size,split,valid_datapoint,width,year
0,train/new_data/000000000001.jpg,81626544,https://dagshub.com/api/v1/repos/lokitek/COCO_...,e66aad884f16045aef55973363872a42d3e189f1729df8...,,480,image/jpeg,159192,train,True,640,2017
1,train/new_data/000000000016.jpg,81626545,https://dagshub.com/api/v1/repos/lokitek/COCO_...,a8a75551047ff8aa81eb407aa4f3e38f96c900aa6affb7...,,640,image/jpeg,230884,train,True,480,2017
2,train/new_data/000000000019.jpg,81626546,https://dagshub.com/api/v1/repos/lokitek/COCO_...,0e3dc4e8d76974e5b96afd0cc0535c405e3c60db0d9867...,,427,image/jpeg,284532,train,True,640,2017
3,train/new_data/000000000057.jpg,81626547,https://dagshub.com/api/v1/repos/lokitek/COCO_...,06b7f21d8bb001ee8c10eedcc16ddfc1851c96ab5dc642...,,427,image/jpeg,189943,train,True,640,2017
4,train/new_data/000000000063.jpg,81626548,https://dagshub.com/api/v1/repos/lokitek/COCO_...,e764a32d8ec6384eb95d2e957d29d148fd2486e27b2da8...,,480,image/jpeg,216259,train,True,640,2017
...,...,...,...,...,...,...,...,...,...,...,...,...
95,train/new_data/000000001363.jpg,81626639,https://dagshub.com/api/v1/repos/lokitek/COCO_...,a76e3ba3b006a9fb3ade44b4d4d944620386b3195a2ee9...,,359,image/jpeg,153605,train,True,500,2017
96,train/new_data/000000001367.jpg,81626640,https://dagshub.com/api/v1/repos/lokitek/COCO_...,769a5677fbe265f5af38d968bb0dbe489d8b19b1ee8d84...,,333,image/jpeg,106330,train,True,500,2017
97,train/new_data/000000001371.jpg,81626641,https://dagshub.com/api/v1/repos/lokitek/COCO_...,b14c664fa4fffaff99b593d54a05fa1460b62419c768e7...,,480,image/jpeg,261056,train,True,640,2017
98,train/new_data/000000001380.jpg,81626642,https://dagshub.com/api/v1/repos/lokitek/COCO_...,909241c0da70890b4a62cdae151092dd3b39cba0430f8d...,,429,image/jpeg,163447,train,True,640,2017


In [41]:
enriched_new_md = new_md.apply(lambda x: dataset_func.create_metadata(x), axis=1)
dagshub.common.config.dataengine_metadata_upload_batch_size = 50
ds.upload_metadata_from_dataframe(enriched_new_md, path_column="path")

AttributeError: 'str' object has no attribute 'decode'

In [ ]:
new_data.visualize()


In [ ]:
new_data.annotate()
